<a href="https://colab.research.google.com/github/Predic-learning/Buiding-Energy/blob/main/Floor_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error
data = pd.read_csv('/content/drive/MyDrive/weather/Fl-5.csv')
#data['Tools'].replace({'AIR':0, 'PC':1 , 'lamp':2}, inplace=True)
data.head()

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower
0,5,AIR,501,1872,38000,3.3,1,3.3,39,128.7
1,5,AIR,502,1873,38000,3.3,1,3.3,39,128.7
2,5,AIR,503,1874,38000,3.3,1,3.3,39,128.7
3,5,AIR,502,1869,38000,3.3,1,3.3,39,128.7
4,5,AIR,503,1870,38000,3.3,1,3.3,39,128.7


In [2]:
num_points = 368
max_error_percent = 20
randomP = [random.uniform(-3.3, 3.3) for _ in range(num_points)]
errorX = [valueX * (max_error_percent / 100) * random.choice([-1, 1]) for valueX in randomP]
dfX = pd.DataFrame({'random_dataX': errorX})
dfX['absolute_random_dataX'] = dfX['random_dataX'].abs()
random_Power = data['PowerHr'] - dfX['absolute_random_dataX']
data = data.assign(random_Power = random_Power)

randomS = [random.uniform(-128.7, 128.7) for _ in range(num_points)]
error = [value * (max_error_percent / 100) * random.choice([-1, 1]) for value in randomS]
df = pd.DataFrame({'random_data': error})
df['absolute_random_data'] = df['random_data'].abs()
random_SumPower = data['SumPower'] - df['absolute_random_data']
data = data.assign(random_SumPower = random_SumPower)
data

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower,random_Power,random_SumPower
0,5,AIR,501,1872,38000,3.30,1,3.300,39,128.70,3.017076,114.340053
1,5,AIR,502,1873,38000,3.30,1,3.300,39,128.70,3.277342,127.158354
2,5,AIR,503,1874,38000,3.30,1,3.300,39,128.70,3.079706,114.517535
3,5,AIR,502,1869,38000,3.30,1,3.300,39,128.70,2.957198,121.636814
4,5,AIR,503,1870,38000,3.30,1,3.300,39,128.70,2.877117,121.959968
5,5,AIR,504,1871,38000,3.30,1,3.300,39,128.70,2.944462,125.479599
6,5,AIR,503,1866,38000,3.30,1,3.300,36,118.80,3.270220,99.912420
7,5,AIR,504,1867,38000,3.30,1,3.300,36,118.80,3.124613,118.770180
8,5,AIR,505,1868,38000,3.30,1,3.300,36,118.80,2.966004,112.225746
9,5,AIR,504,1863,38000,3.30,1,3.300,30,99.00,2.814533,82.480689


In [3]:
#X = data[['PowerHr','time']].values
#y = data['SumPower'].values.reshape(-1, 1)
X = data[['time', 'random_Power']].values
y = data[['random_SumPower']].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
from sklearn.svm import SVR

svr = SVR(kernel='linear', C=1)
svr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svr.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.2f}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

r2 = r2_score(y_test, y_pred)
print(f"Accuracy: {r2 * 100:.2f}%")

MAE: 7.67
MSE: 75.76
RMSE: 8.70
Accuracy: 94.92%


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
T= 1
Data = [[data.loc[T, 'time'], data.loc[T, 'random_Power']]]
y_predict = svr.predict(Data)
y_predict

array([130.60691601])